In [1]:
# LIB UPLOAD CELL:
###########################################

import MAIN_MODULE as mm


path_main = '/Users/stepanvinckevich/Desktop/IMPORTANT NOW/QIS QRL/CODE/qRC/'
path_EVAL = path_main + 'EVALUATION/BRIDGEON_STM_60/'
mm.sys.path.append(path_main)

from matplotlib import pyplot as plt
import matplotlib.font_manager as font_manager 
from matplotlib.pyplot import gca



In [2]:
# EVALUATION CELL:
###########################################

multiplexing = 10
# FUNCTION EVALUATE IMPLEMENT ANALYSIS WITH  BRIDGE SET TO ON !!!
def evaluate_bridge_on(init_state, signal, depolarization_prob):
    # Initialization
    inp = signal[2000:2200]
    result = mm.np.zeros((len(inp), multiplexing, mm.n_qubitsB))
    state = init_state
    # PRECPROCESSING --> IT IS NECESSARY TO SATISFY ECHO STATE PROPERTY. 2000 temporal preproc. steps.
    for i in range(2000):
        state = mm.encoding_step(state, signal[i])
        state = mm.evolution_step(state)
        state = mm.entangled_channel_step(state, depolarization_prob) ## ENTANGLING CHANNEL EXEC FUNCTION
        
    # READOUT  ( TEMPORAL MULTIPLEXING )
    for n in range(len(inp)):
        state = mm.encoding_step(state, inp[n])
        for m in range(multiplexing):
            state = mm.evolution_step(state)
            state = mm.entangled_channel_step(state, depolarization_prob)

            for site in range(mm.n_qubitsB):
                state, outcome = mm.separated_measurement_step(state, site + mm.n_qubitsA)
                result[n, m, site] = outcome

    return result

In [3]:
# EXECUTION CELL:
###########################################
num_signal_samples = 60
num_states_samples = 60

for s in range(num_signal_samples):
    print('Signal sample num:' +  str(s))
    signal = [mm.np.random.choice(2) for _ in range(2200)]
    for i in mm.tqdm(range(num_states_samples)):
        # Initial density matrix
        init_state = mm.qt.tensor(*[mm.qt.rand_ket(2) for _ in range(mm.n_qubits_total)])
        init_state = init_state * init_state.dag()
        init_state = init_state.full()
        results = evaluate_bridge_off(init_state, signal, 0)
        mm.np.save(path_EVAL + str(s) + 'results' + str(i), results)
        mm.np.save(path_EVAL + str(s)+'signal', signal)

Signal sample num:0


NameError: name 'n_qubits_total' is not defined